In this notebook we show how to run a simple deep learning model to classify whale calls. We use the [keras](https://keras.io/) library.

In [18]:
# importing multiple visualization libraries
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
import pylab as pl
import seaborn

In [19]:
# importing libraries to manipulate the data files
import os
from glob import glob

In [20]:
# import a library to read the .aiff format
import aifc

In [21]:
filenames = glob(os.path.join('whale_data','train','*.aiff'))

In [23]:
from scipy import signal

In [25]:
# read signals and apply the welch filter
feature_dict = {}
fs = 2000
for filename in filenames[::1]:
    aiff = aifc.open(filename,'r')
    whale_strSig = aiff.readframes(aiff.getnframes())
    whale_array = np.fromstring(whale_strSig, np.short).byteswap()
    feature = 10*np.log10(signal.welch(whale_array, fs=fs, window='hanning', nperseg=256, noverlap=128+64)[1])
    feature_dict[filename] = feature

/Users/valentina/anaconda/envs/data_mining_test/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  import sys


In [26]:
import pandas as pd
XX = pd.DataFrame(feature_dict)
XX = np.array(XX)
XX.shape

(129, 30000)

In [27]:
# Deep learning on time domain samples.
from __future__ import division
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard

In [12]:
y = y.astype('float32')

In [13]:
from sklearn.model_selection import train_test_split
target_names = ['Upcall', 'NO_Upcall']


X_train, X_test, y_train, y_test = train_test_split(XX.T, y, test_size=0.20, random_state=2018)

# Convert label to onehot
#y_train = keras.utils.to_categorical(y_train, num_classes=2)
#y_test = keras.utils.to_categorical(y_test, num_classes=2)

print(X_train.shape)
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

(24000, 129)


In [14]:
# Build the Neural Network
model = Sequential()

model.add(Conv1D(16, 3, activation='relu', input_shape=(129, 1)))
model.add(Conv1D(16, 3, activation='relu'))
model.add(BatchNormalization())

model.add(Conv1D(32, 3, activation='relu'))
model.add(Conv1D(32, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Conv1D(64, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(BatchNormalization())

model.add(MaxPooling1D(2))
model.add(Conv1D(256, 3, activation='relu'))
model.add(Conv1D(256, 3, activation='relu'))
model.add(BatchNormalization())
model.add(GlobalAveragePooling1D())

model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=SGD(lr=0.01),
              metrics=['accuracy'])




model_name = 'deep_1'
top_weights_path = 'model_' + str(model_name) + '.h5'

callbacks_list = [ModelCheckpoint(top_weights_path, monitor = 'val_acc', verbose = 1, save_best_only = True, save_weights_only = True), 
    EarlyStopping(monitor = 'val_acc', patience = 6, verbose = 1),
    ReduceLROnPlateau(monitor = 'val_acc', factor = 0.1, patience = 3, verbose = 1),
    CSVLogger('model_' + str(model_name) + '.log')]

We see a lot of jargon:


* Max Pooling/ Average Pooling
* Batch normalization
* Epochs
* Adam Optimizer
* Convolutional Layers
* Cross Entropy
* Adam Optimizer
* ReLU
* Batch Size
* Learning Rate

In [ ]:
#from keras.utils import plot_model
#plot_model(model,to_file='model_plot.png')

![new text](model_plot.png)

In [ ]:
%%time
# Fitting the Model (this will take a loooooooot of time)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_test, y_test], callbacks = callbacks_list)

Running with tensorboard to monitoring.

In [ ]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data = [X_test, y_test], callbacks = [tensorboard])

Train on 24000 samples, validate on 6000 samples
Epoch 1/100
24000/24000 [==============================] - 19s 796us/step - loss: 0.4787 - acc: 0.7856 - val_loss: 0.4562 - val_acc: 0.8132
Epoch 2/100
24000/24000 [==============================] - 19s 801us/step - loss: 0.3990 - acc: 0.8258 - val_loss: 0.4488 - val_acc: 0.7990
Epoch 3/100
24000/24000 [==============================] - 22s 914us/step - loss: 0.3690 - acc: 0.8387 - val_loss: 0.3740 - val_acc: 0.8297
Epoch 4/100
24000/24000 [==============================] - 27s 1ms/step - loss: 0.3457 - acc: 0.8475 - val_loss: 0.3489 - val_acc: 0.8395
Epoch 5/100
24000/24000 [==============================] - 21s 894us/step - loss: 0.3332 - acc: 0.8521 - val_loss: 0.3475 - val_acc: 0.8397
Epoch 6/100
24000/24000 [==============================] - 22s 898us/step - loss: 0.3213 - acc: 0.8589 - val_loss: 0.3496 - val_acc: 0.8468
Epoch 7/100
24000/24000 [==============================] - 21s 891us/step - loss: 0.3124 - acc: 0.8630 - val_loss

#### Evaluation

In [ ]:
model.load_weights(top_weights_path)
loss, acc = model.evaluate(X_test, y_test, batch_size=16)
print('Test accuracy:', acc)

In [ ]:
# predict
y_pred = model.predict(X_test)

In [ ]:
sum(y_pred[y_test == 1]>.1)/len(y_test == 1)

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = average_precision_score(y_test.astype('int'), y_pred)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

precision, recall, _ = precision_recall_curve(y_test.astype('int'), y_pred)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))

**References:**

[Deep Learning Glossary](http://www.wildml.com/deep-learning-glossary/)

[Keras and NN Tutorial](https://indico.cern.ch/event/506145/contributions/2132944/attachments/1258124/1858154/NNinKeras_MPaganini.pdf)

Free GPU usage: [Google Colaboratory notebooks](https://colab.research.google.com/notebooks/welcome.ipynb#recent=true) & [Kaggle Kernels](https://www.kaggle.com/kernels).